In [1]:
import os
import json
import glob
import nussl
import museval

from pathlib import Path

In [2]:
class BoogieWoogie(nussl.datasets.BaseDataset):
    def get_items(self, folder):
        path_list = []
        for subdir, dirs, files in os.walk(folder):
            path_list.append(subdir)
        
        items = path_list[1:]
        return items

    def process_item(self, item):
        sources = {}
        #mix = nussl.AudioSignal(item+'/mixture.wav').to_mono()
        sources['left'] = nussl.AudioSignal(item+'/left.wav').to_mono()
        sources['right'] = nussl.AudioSignal(item+'/right.wav').to_mono()
        
        mix = sum(sources.values())
        
        metadata = {
            'labels': ['left', 'right']
        }

        output = {
            'mix': mix,
            'sources': sources,
            'metadata': metadata
        }
        return output

In [3]:
folder = './audio/test/'
stft_params = nussl.STFTParams(window_length=256, hop_length=64, window_type='sqrt_hann')
tfm = nussl.datasets.transforms.Compose([
    nussl.datasets.transforms.PhaseSensitiveSpectrumApproximation(),
    nussl.datasets.transforms.MagnitudeWeights(),
    nussl.datasets.transforms.MagnitudeSpectrumApproximation(),
    #nussl.datasets.transforms.ToSeparationModel()
])
boogie_woogie_dataset = BoogieWoogie(
    folder, stft_params=stft_params, transform=tfm)

In [4]:
def run_alg(alg, song):
    if alg == 'ibm':
        separator = nussl.separation.benchmark.IdealBinaryMask(song['mix'], song['sources'])
    if alg == 'hp':
        separator = nussl.separation.benchmark.HighLowPassFilter(song['mix'], 261.626)
    if alg == 'repetsim':
        separator = nussl.separation.primitive.RepetSim(input_audio_signal=song['mix'], 
                                               similarity_threshold=0.3, min_distance_between_frames=1, 
                                               max_repeating_frames=100, high_pass_cutoff=200, 
                                               mask_type='soft', mask_threshold=0.9)
    if alg == 'repet':
        separator = nussl.separation.primitive.Repet(input_audio_signal=song['mix'], 
                                         min_period=None, max_period=None, 
                                         period=None, high_pass_cutoff=200.0, 
                                         mask_type='soft', mask_threshold=0.9)
    if alg == '2dft':
        separator = nussl.separation.primitive.FT2D(input_audio_signal=song['mix'], 
                                       neighborhood_size=(1, 15), high_pass_cutoff=180.0, 
                                       quadrants_to_keep=(0,1,2,3), filter_approach='local_std', 
                                       use_bg_2dft=True, mask_type='soft', mask_threshold=0.9)
    if alg == 'umx':
        print('Perform source separation using Open-Unmix first! Skipping evaluation...')
        pass
    
    estimates = separator()
    
    return estimates

In [5]:
def run_and_eval(alg, dataset, overwrite=False, eval_method='si'):
    for i in dataset:
        song = i
                
        song_path = str(song['mix'].path_to_input_file[13:-9])
        scores_path = 'scores/' +alg+'/'
        if eval_method == 'si':
            scores_path = 'si-' + scores_path
        estimates_path = 'estimates/' +alg+'/'+song_path+'/'
        print(estimates_path)
        
        if not os.path.exists(scores_path):
            os.makedirs(scores_path)
        if not os.path.exists(estimates_path):
            os.makedirs(estimates_path)
        
        
        if overwrite is False:
            if Path(estimates_path+'left.wav').exists() and Path(estimates_path+'right.wav').exists():
                estimates = []
                estimates.append(nussl.AudioSignal(estimates_path+'left.wav').to_mono())
                estimates.append(nussl.AudioSignal(estimates_path+'right.wav').to_mono())
                estimates[1].path_to_input_file = estimates[0].path_to_input_file
            else:       
                estimates = run_alg(alg, song)
                estimates[0].write_audio_to_file(estimates_path+'left.wav')
                estimates[1].write_audio_to_file(estimates_path+'right.wav')
        
        else:
            estimates = run_alg(alg, song)
            estimates[0].write_audio_to_file(estimates_path+'left.wav')
            estimates[1].write_audio_to_file(estimates_path+'right.wav')
        
        
        true_sources = [song['sources']['left'], song['sources']['right']]
        
        if estimates[0].signal_length > true_sources[0].signal_length:
            # truncate the estimate signals
            diff = estimates[0].signal_length - true_sources[0].signal_length
            
            estimates[0].truncate_samples(estimates[0].signal_length-diff)
            estimates[1].truncate_samples(estimates[1].signal_length-diff)
            
        elif estimates[0].signal_length < true_sources[0].signal_length:
            #perform zero padding on the estimates signals
            diff = true_sources[0].signal_length - estimates[0].signal_length
            
            estimates[0].zero_pad(0, diff)
            estimates[1].zero_pad(0, diff)
         
        
        if eval_method == 'standard':
            evaluate = nussl.evaluation.BSSEvalV4([estimates[0], estimates[1]], true_sources, compute_permutation=True, best_permutation_key='SDR')#, ['left', 'right'])
            scores = evaluate.evaluate()
            
        elif eval_method == 'si':
            evaluate = nussl.evaluation.BSSEvalScale([estimates[0], estimates[1]], true_sources, compute_permutation=True, best_permutation_key='SI-SIR')#, ['left', 'right'])
            scores = evaluate.evaluate()
            
        
        with open(scores_path+song_path+'.json', 'w') as f:
            json.dump(scores, f, indent=2)
        
    return

In [6]:
alg_set = ['ibm', 'hp', 'repet', 'repetsim', '2dft', 'umx']

In [7]:
for alg in alg_set:
    run_and_eval(alg, boogie_woogie_dataset)

estimates/ibm/5. MIDI-Pianos - (1)-Boogie/
estimates/ibm/7. AlLevy - serenade/
estimates/ibm/7. AlLevy - woodchop/
estimates/ibm/4. BitMidi - Boogie-For-Breakfast/
estimates/ibm/5. MIDI-Pianos - (1)-Hazel's Boogie Woogie/
estimates/ibm/7. AlLevy - spivak - bugle/
estimates/ibm/3. TerryButters - shuffle boogie with trad changes (2)/
estimates/ibm/7. AlLevy - rhumb/
estimates/ibm/7. AlLevy - bearcat/
estimates/ibm/3. TerryButters - honeydripper (1)/
estimates/ibm/4. BitMidi - Moo-Cow-Boogie (1)/
estimates/ibm/1. RobRio - SWAN1/
estimates/ibm/7. AlLevy - cafe/
estimates/ibm/7. AlLevy - double/
estimates/ibm/10. RTpress - BSant_sk/
estimates/ibm/1. RobRio - R3OCKHOU/
estimates/ibm/1. RobRio - Cow Cow Blues/
estimates/ibm/7. AlLevy - mlboogie/
estimates/ibm/1. RobRio - Fstyancy/
estimates/ibm/7. AlLevy - prayer/
estimates/ibm/10. RTpress - piggy_sk/
estimates/ibm/7. AlLevy - bounce/
estimates/ibm/4. BitMidi - d.c.boogie/
estimates/ibm/7. AlLevy - roseroom/
estimates/ibm/7. AlLevy - 3blind/


In [8]:
json_files = glob.glob(f"{'si-scores/ibm'}/*.json")
df_ibm = nussl.evaluation.aggregate_score_files(json_files)
df_ibm

,source,file,MIX-SD-SDR,MIX-SI-SDR,MIX-SNR,SD-SDR,SD-SDRi,SI-SAR,SI-SDR,SI-SDRi,SI-SIR,SNR,SNRi,SRR
0,estimates/ibm/7. AlLevy - 3blind/left.wav,7. AlLevy - 3blind.json,-6.283529,-6.283270,-6.422302,11.152237,17.435766,11.222560,11.217971,17.501241,40.981541,11.441867,17.864169,29.394182
1,estimates/ibm/4. BitMidi - The-Lively-Boogie-N...,4. BitMidi - The-Lively-Boogie-Nights.json,-4.991467,-4.990582,-5.213894,8.243317,13.234784,8.394527,8.373505,13.364087,31.535454,8.805649,14.019543,23.493750
2,estimates/ibm/7. AlLevy - bounce/left.wav,7. AlLevy - bounce.json,-7.437467,-7.437453,-7.404342,10.982919,18.420387,11.157285,11.148236,18.585688,37.965293,11.445730,18.850071,25.235793
3,estimates/ibm/4. BitMidi - Moo-Cow-Boogie (1)/...,4. BitMidi - Moo-Cow-Boogie (1).json,-6.639543,-6.639484,-6.714266,11.636843,18.276386,11.706047,11.677359,18.316843,33.492520,11.853906,18.568172,31.935818
4,estimates/ibm/10. RTpress - BSant_sk/left.wav,10. RTpress - BSant_sk.json,-5.709202,-5.709034,-5.813574,10.181302,15.890504,10.289998,10.267529,15.976564,33.141410,10.548224,16.361798,27.300686
5,estimates/ibm/7. AlLevy - bearcat/left.wav,7. AlLevy - bearcat.json,-2.632307,-2.632256,-2.672811,12.560892,15.193198,12.628609,12.625862,15.258118,44.619989,12.806903,15.479714,30.833879
6,estimates/ibm/3. TerryButters - shuffle boogie...,3. TerryButters - shuffle boogie with trad cha...,-5.454919,-5.454909,-5.432099,11.756109,17.211029,11.846299,11.819705,17.274614,33.963089,12.021309,17.453408,30.171889
7,estimates/ibm/4. BitMidi - Cadillac-Avenue-Boo...,4. BitMidi - Cadillac-Avenue-Boogie.json,-6.849362,-6.849207,-6.964352,11.300631,18.149993,11.352098,11.341494,18.190701,37.470155,11.527225,18.491577,31.557788
8,estimates/ibm/7. AlLevy - serenade/left.wav,7. AlLevy - serenade.json,-10.272880,-10.272772,-10.416079,7.815446,18.088326,7.915623,7.914329,18.187101,43.185425,8.329859,18.745937,24.291572
9,estimates/ibm/4. BitMidi - Boogie-For-Breakfas...,4. BitMidi - Boogie-For-Breakfast.json,-7.084295,-7.084294,-7.115832,10.252054,17.336349,10.352310,10.307362,17.391656,30.180922,10.549247,17.665079,29.166496


In [9]:
json_files = glob.glob(f"{'si-scores/hp'}/*.json")
df_hp = nussl.evaluation.aggregate_score_files(json_files)
df_hp

,source,file,MIX-SD-SDR,MIX-SI-SDR,MIX-SNR,SD-SDR,SD-SDRi,SI-SAR,SI-SDR,SI-SDRi,SI-SIR,SNR,SNRi,SRR
0,estimates/hp/7. AlLevy - 3blind/left.wav,7. AlLevy - 3blind.json,-3.823711,-3.823369,-3.945004,-1.358237,2.465475,0.574510,0.559579,4.382948,25.204656,3.243677,7.188681,3.115207
1,estimates/hp/4. BitMidi - The-Lively-Boogie-Ni...,4. BitMidi - The-Lively-Boogie-Nights.json,3.284720,3.289456,3.085962,-7.237641,-10.522361,-2.726240,-2.727167,-6.016623,33.965833,1.857205,-1.228758,-5.340274
2,estimates/hp/7. AlLevy - bounce/left.wav,7. AlLevy - bounce.json,0.186273,0.186817,0.090401,-9.115881,-9.302154,-3.864555,-3.864626,-4.051443,44.074254,1.494108,1.403707,-7.576459
3,estimates/hp/4. BitMidi - Moo-Cow-Boogie (1)/l...,4. BitMidi - Moo-Cow-Boogie (1).json,0.821483,0.822245,0.716274,-7.982633,-8.804117,-3.208054,-3.208652,-4.030897,35.428519,1.695314,0.979040,-6.223099
4,estimates/hp/10. RTpress - BSant_sk/left.wav,10. RTpress - BSant_sk.json,-0.520257,-0.518982,-0.679732,-4.271726,-3.751469,-0.883007,-0.883022,-0.364040,53.751960,2.590833,3.270564,-1.609467
5,estimates/hp/7. AlLevy - bearcat/left.wav,7. AlLevy - bearcat.json,-1.184469,-1.184029,-1.285044,4.654800,5.839269,5.502467,5.502268,6.686297,48.881617,6.567742,7.852787,12.168233
6,estimates/hp/3. TerryButters - shuffle boogie ...,3. TerryButters - shuffle boogie with trad cha...,1.188573,1.189424,1.082210,-5.764702,-6.953275,-1.817060,-1.817104,-3.006528,48.107877,2.195932,1.113722,-3.524882
7,estimates/hp/4. BitMidi - Cadillac-Avenue-Boog...,4. BitMidi - Cadillac-Avenue-Boogie.json,0.642418,0.643197,0.533777,-8.204414,-8.846832,-3.315480,-3.315534,-3.958731,45.740891,1.661322,1.127544,-6.501162
8,estimates/hp/7. AlLevy - serenade/left.wav,7. AlLevy - serenade.json,18.464020,18.470122,18.425111,-21.400699,-39.864719,-10.824808,-10.829494,-29.299616,18.843368,0.342865,-18.082245,-21.002188
9,estimates/hp/4. BitMidi - Boogie-For-Breakfast...,4. BitMidi - Boogie-For-Breakfast.json,-0.107223,-0.106516,-0.220284,-7.641846,-7.534623,-3.016966,-3.017903,-2.911386,33.645227,1.757705,1.977988,-5.805396


In [10]:
json_files = glob.glob(f"{'si-scores/repet'}/*.json")
df_repet = nussl.evaluation.aggregate_score_files(json_files)
df_repet

,source,file,MIX-SD-SDR,MIX-SI-SDR,MIX-SNR,SD-SDR,SD-SDRi,SI-SAR,SI-SDR,SI-SDRi,SI-SIR,SNR,SNRi,SRR
0,estimates/repet/7. AlLevy - 3blind/left.wav,7. AlLevy - 3blind.json,-2.681972,-2.534788,-5.164346,-0.466303,2.215669,0.382364,0.382170,2.916958,43.854580,2.728069,7.892414,7.042497
1,estimates/repet/4. BitMidi - The-Lively-Boogie...,4. BitMidi - The-Lively-Boogie-Nights.json,3.587346,4.572415,0.513297,-3.757015,-7.344361,-1.435274,-1.582018,-6.154433,13.203217,2.120569,1.607273,0.288425
2,estimates/repet/7. AlLevy - bounce/left.wav,7. AlLevy - bounce.json,-1.885986,-1.689347,-4.518094,-2.819237,-0.933252,-0.921989,-0.939601,0.749747,22.988751,2.382132,6.900226,1.723854
3,estimates/repet/4. BitMidi - Moo-Cow-Boogie (1...,4. BitMidi - Moo-Cow-Boogie (1).json,1.399769,1.920070,-1.526328,-3.865275,-5.265044,-1.239395,-1.363744,-3.283814,14.129618,2.295747,3.822075,-0.278610
4,estimates/repet/10. RTpress - BSant_sk/left.wav,10. RTpress - BSant_sk.json,1.041613,1.320354,-1.129337,-3.882925,-4.924538,-1.493859,-1.509025,-2.829379,23.067114,2.201239,3.330576,-0.126666
5,estimates/repet/7. AlLevy - bearcat/left.wav,7. AlLevy - bearcat.json,-2.289425,-2.105799,-4.961680,0.620599,2.910024,0.900816,0.633001,2.738800,12.865741,1.042040,6.003720,26.069986
6,estimates/repet/3. TerryButters - shuffle boog...,3. TerryButters - shuffle boogie with trad cha...,3.018587,3.456990,0.872701,-5.100646,-8.119233,-2.232662,-2.258059,-5.715048,20.085263,1.930615,1.057914,-1.915886
7,estimates/repet/4. BitMidi - Cadillac-Avenue-B...,4. BitMidi - Cadillac-Avenue-Boogie.json,-0.737350,-0.305071,-4.281853,-0.304218,0.433132,1.105578,0.895922,1.200993,14.163067,3.268918,7.550770,5.867569
8,estimates/repet/7. AlLevy - serenade/left.wav,7. AlLevy - serenade.json,-0.263315,0.254361,-3.945545,-10.553170,-10.289855,-6.254396,-6.268368,-6.522729,18.663479,0.737784,4.683329,-8.526971
9,estimates/repet/4. BitMidi - Boogie-For-Breakf...,4. BitMidi - Boogie-For-Breakfast.json,2.432885,2.637339,0.897170,-8.544828,-10.977713,-4.132326,-4.134232,-6.771571,29.447436,1.382511,0.485341,-6.591757


In [11]:
json_files = glob.glob(f"{'si-scores/repetsim'}/*.json")
df_repetsim = nussl.evaluation.aggregate_score_files(json_files)
df_repetsim

,source,file,MIX-SD-SDR,MIX-SI-SDR,MIX-SNR,SD-SDR,SD-SDRi,SI-SAR,SI-SDR,SI-SDRi,SI-SIR,SNR,SNRi,SRR
0,estimates/repetsim/7. AlLevy - 3blind/left.wav,7. AlLevy - 3blind.json,-2.856197,-2.657996,-5.873299,0.658462,3.514659,1.125146,1.120832,3.778828,31.151705,2.901132,8.774431,10.615531
1,estimates/repetsim/4. BitMidi - The-Lively-Boo...,4. BitMidi - The-Lively-Boogie-Nights.json,-1.067592,-0.313338,-6.284122,2.374593,3.442185,2.496521,2.374662,2.688000,17.954699,2.350259,8.634382,51.064217
2,estimates/repetsim/7. AlLevy - bounce/left.wav,7. AlLevy - bounce.json,-1.819192,-1.527378,-5.096706,-2.181817,-0.362625,-0.697441,-0.727759,0.799618,20.847960,2.353929,7.450634,3.276974
3,estimates/repetsim/4. BitMidi - Moo-Cow-Boogie...,4. BitMidi - Moo-Cow-Boogie (1).json,-0.506440,0.095127,-4.676059,0.271112,0.777552,1.278732,1.052172,0.957045,13.990988,3.151659,7.827718,8.106747
4,estimates/repetsim/10. RTpress - BSant_sk/left...,10. RTpress - BSant_sk.json,-0.242901,0.298966,-4.014124,-0.983531,-0.740631,-0.143170,-0.178470,-0.477436,20.739462,2.307469,6.321593,6.732336
5,estimates/repetsim/7. AlLevy - bearcat/left.wav,7. AlLevy - bearcat.json,-1.561415,-1.216027,-5.052164,2.072531,3.633946,2.184505,2.093214,3.309241,18.912292,1.587594,6.639758,25.303923
6,estimates/repetsim/3. TerryButters - shuffle b...,3. TerryButters - shuffle boogie with trad cha...,1.302391,2.049451,-2.335480,-2.835469,-4.137861,-1.530173,-1.547997,-3.597448,22.328217,1.783890,4.119369,3.072850
7,estimates/repetsim/4. BitMidi - Cadillac-Avenu...,4. BitMidi - Cadillac-Avenue-Boogie.json,-1.801818,-1.321651,-6.211426,2.885818,4.687637,3.471215,3.109673,4.431324,14.085486,4.180817,10.392243,15.875357
8,estimates/repetsim/7. AlLevy - serenade/left.wav,7. AlLevy - serenade.json,2.605359,4.410062,-2.307691,-10.863243,-13.468602,-6.117943,-6.304503,-10.714565,7.458037,0.768495,3.076186,-8.992071
9,estimates/repetsim/4. BitMidi - Boogie-For-Bre...,4. BitMidi - Boogie-For-Breakfast.json,0.477319,1.130704,-3.314626,-4.395295,-4.872614,-2.476914,-2.498182,-3.628886,20.612636,1.567658,4.882284,0.115679


In [12]:
json_files = glob.glob(f"{'si-scores/2dft'}/*.json")
df_2dft = nussl.evaluation.aggregate_score_files(json_files)
df_2dft

,source,file,MIX-SD-SDR,MIX-SI-SDR,MIX-SNR,SD-SDR,SD-SDRi,SI-SAR,SI-SDR,SI-SDRi,SI-SIR,SNR,SNRi,SRR
0,estimates/2dft/7. AlLevy - 3blind/left.wav,7. AlLevy - 3blind.json,-2.634913,-2.209961,-7.269682,-0.321668,2.313245,-0.171380,-0.171383,2.038579,63.715086,1.199365,8.469047,14.361903
1,estimates/2dft/4. BitMidi - The-Lively-Boogie-...,4. BitMidi - The-Lively-Boogie-Nights.json,0.658126,2.356741,-5.852607,0.189355,-0.468771,0.428709,0.229073,-2.127668,13.704156,0.964612,6.817219,20.596867
2,estimates/2dft/7. AlLevy - bounce/left.wav,7. AlLevy - bounce.json,-1.006390,-0.066841,-6.944803,-0.968642,0.037748,-0.388764,-0.536279,-0.469438,14.226680,1.600300,8.545103,9.265095
3,estimates/2dft/4. BitMidi - Moo-Cow-Boogie (1)...,4. BitMidi - Moo-Cow-Boogie (1).json,-0.230927,1.068635,-6.646784,0.433558,0.664485,0.876265,0.562730,-0.505905,12.133452,1.744058,8.390842,15.764060
4,estimates/2dft/10. RTpress - BSant_sk/left.wav,10. RTpress - BSant_sk.json,1.010536,2.138912,-3.808231,-3.188072,-4.198608,-2.268902,-2.277552,-4.416465,24.733117,1.043727,4.851958,4.044150
5,estimates/2dft/7. AlLevy - bearcat/left.wav,7. AlLevy - bearcat.json,-0.668007,-0.079535,-4.882556,2.697057,3.365064,2.771593,2.766807,2.846342,32.348089,1.853704,6.736260,20.674365
6,estimates/2dft/3. TerryButters - shuffle boogi...,3. TerryButters - shuffle boogie with trad cha...,2.315181,3.936528,-2.530457,-3.787843,-6.103023,-2.681651,-2.697110,-6.633638,21.796126,0.967493,3.497950,2.746788
7,estimates/2dft/4. BitMidi - Cadillac-Avenue-Bo...,4. BitMidi - Cadillac-Avenue-Boogie.json,-0.005467,1.482488,-6.730291,0.304204,0.309671,0.935275,0.455816,-1.026672,10.263686,1.733381,8.463672,14.950182
8,estimates/2dft/7. AlLevy - serenade/left.wav,7. AlLevy - serenade.json,1.766709,4.209408,-4.887469,-9.458214,-11.224924,-6.201397,-6.364616,-10.574023,7.966771,0.426869,5.314338,-6.529648
9,estimates/2dft/4. BitMidi - Boogie-For-Breakfa...,4. BitMidi - Boogie-For-Breakfast.json,1.438804,2.719008,-3.415350,-5.557849,-6.996653,-3.802166,-3.811845,-6.530853,22.711623,0.849331,4.264680,-0.756685


In [13]:
json_files = glob.glob(f"{'si-scores/umx'}/*.json")
df_umx = nussl.evaluation.aggregate_score_files(json_files)
df_umx

,source,file,MIX-SD-SDR,MIX-SI-SDR,MIX-SNR,SD-SDR,SD-SDRi,SI-SAR,SI-SDR,SI-SDRi,SI-SIR,SNR,SNRi,SRR
0,estimates/umx/7. AlLevy - 3blind/left.wav,7. AlLevy - 3blind.json,-6.533132,-6.524079,-7.417603,2.912971,9.446103,8.039234,7.998737,14.522816,28.322344,-4.915475,2.502128,4.524978
1,estimates/umx/4. BitMidi - The-Lively-Boogie-N...,4. BitMidi - The-Lively-Boogie-Nights.json,-4.232181,-4.193743,-5.673113,2.906953,7.139134,7.426070,7.425714,11.619456,48.304586,-4.527932,1.145180,4.804675
2,estimates/umx/7. AlLevy - bounce/left.wav,7. AlLevy - bounce.json,-6.892544,-6.879544,-8.011872,3.595818,10.488362,10.277612,10.275044,17.154589,42.557449,-4.058462,3.953410,4.646086
3,estimates/umx/4. BitMidi - Moo-Cow-Boogie (1)/...,4. BitMidi - Moo-Cow-Boogie (1).json,-6.145755,-6.138439,-6.900384,10.262560,16.408315,10.265130,10.264038,16.402477,46.260653,10.312332,17.212716,44.811696
4,estimates/umx/10. RTpress - BSant_sk/left.wav,10. RTpress - BSant_sk.json,-5.477163,-5.463665,-6.437232,3.271530,8.748693,8.965822,8.953869,14.417534,34.563110,-4.392464,2.044768,4.639222
5,estimates/umx/7. AlLevy - bearcat/left.wav,7. AlLevy - bearcat.json,-2.591436,-2.569627,-3.462003,3.478597,6.070034,10.335453,10.323676,12.893303,35.997646,-4.408920,-0.946917,4.484824
6,estimates/umx/3. TerryButters - shuffle boogie...,3. TerryButters - shuffle boogie with trad cha...,-5.163616,-5.156043,-5.846847,3.721627,8.885243,10.862697,10.857418,16.013461,40.012774,-3.920066,1.926781,4.655000
7,estimates/umx/4. BitMidi - Cadillac-Avenue-Boo...,4. BitMidi - Cadillac-Avenue-Boogie.json,-6.598181,-6.593184,-7.251799,11.040903,17.639084,11.046169,11.045535,17.638718,49.386244,10.962420,18.214219,40.919910
8,estimates/umx/7. AlLevy - serenade/left.wav,7. AlLevy - serenade.json,-8.729598,-8.701959,-10.861210,4.691760,13.421357,5.725230,5.707185,14.409145,29.531393,2.005907,12.867116,11.502991
9,estimates/umx/4. BitMidi - Boogie-For-Breakfas...,4. BitMidi - Boogie-For-Breakfast.json,-6.228128,-6.215106,-7.260659,2.947362,9.175489,7.103435,7.098375,13.313481,36.435647,-4.159380,3.101279,5.055305


In [14]:
df_ibm.to_csv('si-csvs/ibm.csv', index=False)
df_hp.to_csv('si-csvs/hp.csv', index=False)
df_repet.to_csv('si-csvs/repet.csv', index=False)
df_repetsim.to_csv('si-csvs/repetsim.csv', index=False)
df_2dft.to_csv('si-csvs/2dft.csv', index=False)
df_umx.to_csv('si-csvs/umx.csv', index=False)